In [459]:
import numpy as np
from matplotlib import pyplot as plt
import numba as nb
%matplotlib inline

In [608]:
n_pixels = 2
rate_pixel = 1  # Hz
duration = 1000  # seconds
n_samples = 100000
sample_width = duration / n_samples
time_axis = np.arange(0, duration, sample_width)
n_repeats = 10000
n_counts_per_pixel = int(duration * rate_pixel)

times = np.random.uniform(0, duration, (n_pixels, n_counts_per_pixel))
#times = np.sort(times, axis=1)

In [599]:
within = 1
@nb.njit()
def get_coincidence_counts(n_counts_per_pixel):
    n_coincidence = 0
    for ipix in range(n_pixels):
        for jpix in range(n_pixels):
            if ipix == jpix:
                continue
            for ti in times[ipix]:
                for tj in times[jpix]:
                    if (tj >= ti) & (tj < (ti + within)):
                        n_coincidence += 1
                        
    return n_coincidence

In [600]:
n = np.zeros(n_repeats)
for i in range(n_repeats):
    n[i] = get_coincidence_counts(n_counts_per_pixel)
rate = n.mean(0) / duration
print(f"{rate:.07f}")

2.0630000


In [787]:
n_pixels = 2
rate_pixel = 0.1  # Hz
duration = 1000  # seconds
n_samples = 100
sample_width = duration / n_samples
time_axis = np.arange(0, duration, sample_width)
n_repeats = 1000
n_counts_per_pixel = int(duration * rate_pixel)

In [788]:
@nb.njit()
def get_coincidence_counts(n_counts_per_pixel):
    times = np.random.uniform(0, duration, (n_pixels, n_counts_per_pixel))
    n_coincidence = 0
    for ipix in range(n_pixels):
        for jpix in range(n_pixels):
            if ipix == jpix:
                continue
            for ti in times[ipix]:
                si = int(ti / sample_width)
                for tj in times[jpix]:
                    sj = int(tj / sample_width)
                    if (si == sj):
                        n_coincidence += 1
                        
    return n_coincidence

In [683]:
n_pixels = 2
rate_pixel = 0.1  # Hz
duration = 1000  # seconds
n_samples = 1000
sample_width = duration / n_samples
time_axis = np.arange(0, duration, sample_width)
n_repeats = 1000
n_counts_per_pixel = int(duration * rate_pixel)

In [684]:
rate_pixel_measured = np.zeros((n_repeats, n_pixels))
rate_coincidence_measured = np.zeros(n_repeats)
for i in range(n_repeats):
    times = np.random.uniform(0, duration, (n_pixels, n_counts_per_pixel))
    times_samples = (times / sample_width).astype(np.int)
    trigger_line = np.zeros((n_pixels, time_axis.size), dtype=np.bool)
    trigger_line[np.arange(n_pixels)[:, None], times_samples] = True
    rate_pixel_measured[i] = trigger_line.sum(1) / duration
    coincidence = trigger_line.all(0)
    coincidence = trigger_line[:, coincidence].sum(0)
    n_coincidence = coincidence.sum()
    rate_coincidence_measured[i] = n_coincidence / duration
    
rate_pixel_measured_avg = rate_pixel_measured.mean(0)
rate_pixel_measured_std = rate_pixel_measured.std(0)
rate_coincidence_measured_avg = rate_coincidence_measured.mean()
rate_coincidence_measured_std = rate_coincidence_measured.std()

In [ ]:
camera = Camera(
    mapping=SSTCameraMapping(n_pixels=8), 
    digital_trigger_length=8,
    continuous_readout_duration=128,
)
acquisition = EventAcquisition(camera=camera)

n_continuous_samples = camera.continuous_readout_time_axis.size
s0 = n_continuous_samples // 2
n_triggers = np.zeros(n_continuous_samples)
dt = np.zeros(n_continuous_samples)
for s1 in range(n_continuous_samples):
    digital_trigger = np.zeros((camera.mapping.n_superpixels, n_continuous_samples), dtype=np.bool)
    digital_trigger[0, s0] = True
    digital_trigger[1, s1] = True
    extended = acquisition.trigger.extend_by_digital_trigger_length(digital_trigger)
    n_triggers[s1] = acquisition.trigger.get_backplane_trigger(extended).size
    dt[s1] = (s0 - s1) * camera.continuous_readout_sample_width
    
plt.plot(dt, n_triggers)
plt.xlim(-10, 10)